# 0. Initialize

In [0]:
# Specify a working directory
workdir = '/content/drive/My Drive/test_eclaire'

In [0]:
# Mount Google Drive to store FITS files to use.
from google.colab import drive
drive.mount('/content/drive')

In [0]:
import sys
import os
import shutil
import requests
import functools
import warnings
from glob import glob

from astropy.io import fits
from astropy.wcs import WCS
import cupy  as cp
import numpy as np

import matplotlib.pyplot as plt
from astropy.visualization import ZScaleInterval
from tqdm.notebook import tqdm

In [ ]:
# Install Ecliare
!pip install git+https://github.com/MNiwano/Eclaire

In [0]:
from eclaire import FitsContainer, reduction, fixpix, imalign, imcombine

In [0]:
fitslist = ['raw{:02d}.fits'.format(i) for i in range(10)]

dark = 'dark.fits'
flat = 'flat.fits'
bpmask = 'bpmask.fits'

output = 'combine.fits'

In [0]:
# Create the working directory on Google Drive and download the test data-set.
if not os.path.exists(workdir):
  os.mkdir(workdir)

os.chdir(workdir)

url = 'http://sncwall.hp.phys.titech.ac.jp:2388/samplefits/'
for f in tqdm(fitslist+[dark,flat,bpmask]):
    if not os.path.exists(f):
        res = requests.get(url+f, stream=True)
        with open(f, 'wb') as fp:
            shutil.copyfileobj(res.raw, fp)

  # 1. Data Loading
<b>eclaire.FitsContainer</b> class is used to load FITS images for reduction processing. When a list of file names is given, FITS in the list are searched for in the current directory and loaded. The header is stored in <b>FitsContainer.header</b> as a <b>list</b> of <b>astropy.io.fits.Header</b> instances, and the image data is stored in <b>FitsContainer.data</b> as a 3D <b>cupy.ndarray</b>. Each image data is stacked in the direction of the first axis, with the second axis corresponding to the Y-axis and the third axis to the X-axis.

In [0]:
fc = FitsContainer(
    fitslist,
    #wrapper=functools.partial(tqdm,total=len(fitslist)), # Display progress bar
)

# Removal of overscan areas in the image
fc.data = fc.data[:,2:1022,52:1072]

In [0]:
npdark = fits.getdata(dark)
npflat = fits.getdata(flat)
npbpm = fits.getdata(bpmask)

cpdark = cp.asarray(npdark,dtype='float32')
cpflat = cp.asarray(npflat+(npflat==0.0),dtype='float32')
cpbpm = cp.asarray(npbpm,dtype='float32')

# 2. Calculate shift amount
You can find the relative positions of the images with using WCS information in headers. Eclaire is not used here.

In [0]:
# For ignoreing FITSFixedWarning
with warnings.catch_warnings():
    warnings.simplefilter('ignore',FITSFixedWarning)
    
    wcs = [WCS(header) for header in fc.header]

xy0 = np.array([[450,450],[450,650],[650,450],[650,650]])
ad0 = wcs[0].wcs_pix2world(xy0,1)
shift = np.empty([len(wcs),2],dtype='f4')
for i, w in enumerate(wcs):
    xy = w.wcs_world2pix(ad0,1)
    shift[i] = (xy0-xy).mean(axis=0)

# 3. Reduction
First, <b>eclaire.reduction</b> performs bias & dark subtraction, and flat fielding at once.
This function is equivalent to the following expression in this example.
```
fc.data = (fc.data - cpbias - cpdark) / cpflat
```
However, it can be executed with less overhead of calling and memory usage.
Each input value must be broadcastable.
(This is a reason of increasing the dimension of <b>cpbias</b> with <b>reshape</b> method.)
The next step is correcting bad pixels with using <b>eclaire.firepix</b>.
This function overwrites the bad pixels with the average of the surrounding pixel counts.
The given bad-pixel mask must have non-zero values for the bad-pixel positions and zero otherwise.

In [0]:
# Here we'll use the bias count written in the header,
# but If you want to use the master bias frame,
# just load it as cupy.ndarray in the same way as dark.
bias = [f['PEDLEVEL'] for f in fc.header]
cpbias = cp.array(bias,dtype='f4').reshape(-1,1,1)

In [0]:
fc.data = reduction(fc.data, cpbias, cpdark, cpflat)

In [0]:
fc.data = fixpix(fc.data, cpbpm)

# 4. Align
Align the images based on the relative positions calculated earlier.
<b>shift</b> must be two-dimensional array, where the first value in the second axis direction is interpreted as the X coordinate and the second as the Y coordinate.
The order in the first axis must be the same as the order of the images in the array given as the first argument.

The default interpolation algorithm for sub-pixel shift is bicubic-spline.

In [0]:
fc.data = imalign(
    fc.data, shift,
    #interp='spline3', # specify the interpolation algorithm
    # 'neighbor', 'linear', and 'poly3' can also be selected.
)

# 5. Co-adding
Co-add the images. The default algorithm is sigma-clipped-mean.

In [0]:
combined = imcombine(
    fc.data, name=output, list=fitslist, overwrite=True,
    #combine='mean' # specify the co-adding method
    #width=3.0 # specify the clipping width
    #iters=5 # specify the number of iterations
)

# 6. Show Result

In [0]:
data = fits.getdata(output)
vrange = ZScaleInterval().get_limits(data)
data.clip(*vrange,out=data)

plt.figure()
plt.imshow(data,origin='lower')
plt.colorbar()
plt.show()